In [1]:
import pandas as pd
import numpy as np

In [2]:
df_main = pd.read_csv('clean_data.csv')
df_imdb = pd.read_csv('clean_imdb.csv')

In [3]:
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, RidgeClassifier, RidgeClassifierCV
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB, ComplementNB

In [4]:
np.random.seed(42)
pdf_main = df_main.reindex(np.random.permutation(df_main.index))
pdf_imdb = df_imdb.reindex(np.random.permutation(df_imdb.index))

In [5]:
X_train, X_test, y_train, y_test = pdf_imdb['clean_text'], pdf_main['clean_text'], pdf_imdb['class'], pdf_main['class']

In [6]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Flatten, Conv1D, MaxPooling1D, Dropout, Bidirectional
from keras.optimizers import Adam

Using TensorFlow backend.


In [7]:
import os
embeddings_index = {}
f = open(os.path.join('../', 'glove.6B.200d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [8]:
tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(X_train)

seq_train = tokenizer.texts_to_sequences(X_train)
seq_test = tokenizer.texts_to_sequences(X_test)

In [9]:
np.random.seed(42)

In [17]:
EMBEDDING_DIM = 200
MAX_SEQUENCE_LENGTH = 150

In [18]:
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [19]:
embedding_layer = Embedding(len(tokenizer.word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [20]:
pad_train = pad_sequences(seq_train, maxlen=MAX_SEQUENCE_LENGTH)
pad_test = pad_sequences(seq_test, maxlen=MAX_SEQUENCE_LENGTH)

In [21]:
model = Sequential()
model.add(embedding_layer)
model.add(Bidirectional(LSTM(64, dropout=.25, recurrent_dropout=.25, return_sequences=True)))
model.add(Bidirectional(LSTM(64, dropout=.25, recurrent_dropout=.25, return_sequences=True)))
model.add(Bidirectional(LSTM(64, dropout=.25, recurrent_dropout=.25, return_sequences=False)))
# model.add(BLSTM(128, dropout=.25, recurrent_dropout=.25, return_sequences=True))
# model.add(LSTM(128, dropout=.25, recurrent_dropout=.25))
model.add(Dense(100, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

W0726 16:42:19.948131 140124422502208 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 16:42:19.976829 140124422502208 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0726 16:42:20.070345 140124422502208 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0726 16:42:20.082139 140124422502208 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please us

In [22]:
np.random.seed(42)
model.fit(pad_train, y_train, epochs=5, batch_size=64, validation_data=(pad_test, y_test))

Train on 50000 samples, validate on 2000 samples
Epoch 1/5
50000/50000 [==============================] - 2553s 51ms/step - loss: 0.4116 - acc: 0.8123 - val_loss: 0.3629 - val_acc: 0.8480
Epoch 2/5
50000/50000 [==============================] - 2500s 50ms/step - loss: 0.2384 - acc: 0.9111 - val_loss: 0.4029 - val_acc: 0.8355
Epoch 3/5
 3520/50000 [=>............................] - ETA: 38:37 - loss: 0.1666 - acc: 0.9457

E0726 18:10:09.722599 140124422502208 ultratb.py:149] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3291, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-70705dbd012a>", line 2, in <module>
    model.fit(pad_train, y_train, epochs=5, batch_size=64, validation_data=(pad_test, y_test))
  File "/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/engine/training.py", line 1039, in fit
    validation_steps=validation_steps)
  File "/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/engine/training_arrays.py", line 199, in fit_loop
    outs = f(ins_batch)
  File "/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py", line 2715, in __call__
    return self._call(inputs)
  File "/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py", line 2675, in _call
    fetched = self._callable_fn(*array_vals)
  F

KeyboardInterrupt: 

In [24]:
model.save('very_deep_rnn_embed_trainable.hdf5')

In [23]:
accuracy_score(y_test, model.predict_classes(pad_test))

0.8365